In [ ]:
from IPython.display import HTML, display
from tqdm import tqdm

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
!pip install transformers

from transformers import AutoTokenizer, AutoModel
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.2 MB/s eta 0:00:00


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

In [ ]:
from pathlib import Path
from bs4 import BeautifulSoup
import requests
from bs4.element import Comment
import urllib.request

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def parse(url):
    body = urllib.request.urlopen(url)
    soup = BeautifulSoup(body, "html.parser")
    texts = soup.findAll(string=True)
    visible_texts = filter(tag_visible, texts)
    return "\p ".join(t.strip() for t in visible_texts if len(t.strip())>0)

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import os
import warnings

warnings.filterwarnings('ignore')

xml_file = 'test_data.xml'

output_df = pd.DataFrame(columns=['question', 'answer', 'context'])
qa_dict = {}
tree = ET.parse(xml_file)
root = tree.getroot()
for qa_pair in root.findall("QAPairs")[0]:
  qa_dict['question'] = qa_pair.find('Question').text
  qa_dict['answer'] = qa_pair.find('Answer').text
  url = root.attrib['url']
  qa_dict['context'] = parse(url)
  output_df = output_df.append(qa_dict, ignore_index=True)

In [ ]:
def get_span(refined_text, answers_all):

    answers = []
    for answer in answers_all:
        answer = answer.replace("\n", " ")
        answer = answer.replace("\p"," ")
        answer = answer.strip()
        answers.append(answer)

    refined_answers = []

    spans = []
    for answer in answers:
        answer = answer.replace('-',' ')
        refined_answer = " ".join(answer.split())
        refined_answers.append(refined_answer)
        result = refined_text.find(refined_answer[:50])
        if(result!=-1):
            i, j = result, result + len(refined_answer)
            spans.append((i,j))
        else:
            spans.append((-1,-1))
    return spans, refined_text, refined_answers


text = output_df['context'].iloc[0]
answers_all = output_df['answer'].tolist()

text = text.replace("\n", " ")
text = text.replace("\p"," ")
text = text.strip()
text = text.replace('-',' ')
refined_text = " ".join(text.split())

spans, refined_text, refined_answers = get_span(refined_text, answers_all)

span_df = output_df.copy()

span_df = span_df.join(pd.DataFrame(
    {
        'span_start': -1,
        'span_end': -1,
    }, index=span_df.index
))

for i in range(len(spans)):
    span = spans[i]
    span_df["answer"].iloc[i] = refined_answers[i]
    span_df["context"].iloc[i] = refined_text
    span_df["span_start"].iloc[i] = span[0]
    span_df["span_end"].iloc[i] = span[1]

span_df = span_df[span_df["span_start"]!=-1]
span_df = span_df[span_df["span_end"]!=-1]
span_df = span_df.dropna()

In [ ]:
df = pd.read_csv("data_withNeg1k-5Cols.csv")
df = df.dropna()
print(df)

                                               question  \
0          What is (are) keratoderma with woolly hair ?   
1     How many people are affected by keratoderma wi...   
2           Is keratoderma with woolly hair inherited ?   
3                     What is (are) Knobloch syndrome ?   
4     How many people are affected by Knobloch syndr...   
...                                                 ...   
3463         What is (are) tuberous sclerosis complex ?   
3464          Is tuberous sclerosis complex inherited ?   
3465  What is (are) ornithine transcarbamylase defic...   
3466  What are the genetic changes related to ornith...   
3467  Is ornithine transcarbamylase deficiency inher...   

                                                 answer  \
0     Keratoderma with woolly hair is a group of rel...   
1     Keratoderma with woolly hair is rare; its prev...   
2     Most cases of keratoderma with woolly hair hav...   
3     Knobloch syndrome is a rare condition characte...

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm

all_data = []

for index, row in df.iterrows():
    sentences = []
    sentences.append(row['question'])
    sentences.append(row['context'])
    sentences.append(row['neg_context0'])
    sentences.append(row['neg_context1'])
    sentences.append(row['neg_context2'])
    sentences.append(row['neg_context3'])
    sentences.append(row['neg_context4'])
    all_data.append(sentences)

scores = []
count = 0
for sentences in tqdm(all_data[:200]):
    try:
        tokens = {'input_ids': [], 'attention_mask': []}

        for sentence in sentences:
            new_tokens = tokenizer.encode_plus(sentence, max_length=512,
                                              truncation=True, padding='max_length',
                                              return_tensors='pt')
            tokens['input_ids'].append(new_tokens['input_ids'][0])
            tokens['attention_mask'].append(new_tokens['attention_mask'][0])

        tokens['input_ids'] = torch.stack(tokens['input_ids'])
        tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

        outputs = model(**tokens)
        embeddings = outputs.last_hidden_state
        attention_mask = tokens['attention_mask']
        mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
        masked_embeddings = embeddings * mask
        summed = torch.sum(masked_embeddings, 1)
        summed_mask = torch.clamp(mask.sum(1), min=1e-9)
        mean_pooled = summed / summed_mask
        mean_pooled = mean_pooled.detach().numpy()

        cos_sim = cosine_similarity(
            [mean_pooled[0]],
            mean_pooled[1:]
        )[0]
        check_val = np.where(cos_sim == max(cos_sim))[0][0]
        if(check_val!=0):
          count += 1
          if(count < 10):
              print("Predicted: ", sentences[1+check_val])
              print("Ground Truth: ", sentences[1])
        scores.append(np.where(cos_sim == max(cos_sim))[0][0])
    except:
        continue

In [ ]:
print(count)
file = open('scores.txt','w')
for item in scores:
    file.write(str(item) + "\n")
file.close()

In [ ]:
with open("scores.txt", "r") as f:
	  vals = f.readlines()

scores = []
for val in vals:
    scores.append(int(val.replace('\n', '')))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm


all_data = []

for index, row in df.iterrows():
    sentences = []
    sentences.append(row['question'])
    sentences.append(row['context'])
    sentences.append(row['neg_context0'])
    sentences.append(row['neg_context1'])
    sentences.append(row['neg_context2'])
    sentences.append(row['neg_context3'])
    sentences.append(row['neg_context4'])
    all_data.append(sentences)

count = 0
i = 0
for sentences in tqdm(all_data[:200]):
    if(scores[i]!=0 and i < 10):
        print("Question: ", sentences[0])
        print(":::::::::::::::::::::::::")
        print("Ground Truth Paragraph: ", sentences[1])
        print("-------------------------")
        print("Predicted Paragraph: ", sentences[1+scores[i]])
        print(".........................")
        print(".........................")

    i += 1

100%|██████████| 200/200 [00:00<00:00, 70754.12it/s]

Question:  What is (are) keratoderma with woolly hair ?
:::::::::::::::::::::::::
Ground Truth Paragraph:  Skip navigation National Library of Medicine Menu Health Topics Drugs & Supplements Genetics Medical Tests Medical Encyclopedia About MedlinePlus Search Search MedlinePlus GO About MedlinePlus What's New Site Map Customer Support Health Topics Drugs & Supplements Genetics Medical Tests Medical Encyclopedia You Are Here: Home → Genetics → Genetic Conditions → Keratoderma with woolly hair URL of this page: https://medlineplus.gov/genetics/condition/keratoderma with woolly hair/ Keratoderma with woolly hair To use the sharing features on this page, please enable JavaScript. Description  Keratoderma with woolly hair is a group of related conditions that affect the skin and hair and in many cases increase the risk of potentially life threatening heart problems. People with these conditions have hair that is unusually coarse, dry, fine, and tightly curled. In some cases, the hair is als